# Radial Basis Function

Σκοπός της παρούσας άσκησης είναι η πρόβλεψη ως προς το αν ένας/μία πελάτης/ισσα μίας τράπεζας θα κάνει τη χρήση της προθεσμιακής κατάθεσης ή όχι. Αυτή η πρόβλεψη θα εξαχθεί με βάση συγκεκριμένα δημογραφικά χαρακτηριστικά του/της
όπως η ηλικία, η εργασία, η εκπαίδευση, η οικογενειακή κατάσταση κ.λπ.
Για να το επιτύχουμε αυτό θα κάνουμε χρήση μίας εφαρμογής βασισμένη σε RBF. Στο κρυφό επίπεδο των νευρώνων υπολογίζονται τα κέντρα των clusters με τη χρήση του αλγορίθμου k-means.


 

**Βήμα 1ο**: Αρχικά φορτώνουμε τις απαράιτητες βιβλιοθήκες

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import math
from sklearn.metrics import accuracy_score

**Βήμα 2ο**: διάβαζουμε το αρχείο bank-full.csv και απομονώνουμε τις στήλες με τα χαρακτηριστικά που θα χρησιμοποιήσουμε (εργασία, αν έχει δανειακές υποχρεώσεις κλπ). Το αρχείο είναι διαθέσιμο [εδώ](https://raw.githubusercontent.com/nkostopoulos/StochasticsLabPublic/master/lab9/bank-full.csv) και θα πρέπει να το φορτώσετε στο colab.

In [ ]:
dataset = pd.read_csv('bank-full.csv',sep=None)

In [ ]:
dataset.head()

In [5]:
col_to_use = ['age','balance','day','duration','campaign','pdays','previous']
data = dataset.drop(col_to_use,axis=1)

In [ ]:
data.head()

In [7]:
data = data.apply(LabelEncoder().fit_transform)

In [ ]:
data.head()

In [ ]:
data['job'].unique()

In [ ]:
dataset['job'].unique()

In [ ]:
data_rest = dataset[col_to_use]
data_rest.head()

In [12]:
dataset2 = pd.concat([data_rest,data],axis=1)

In [ ]:
dataset2.head()

**Βήμα 3ο**: Σπάμε στο dataset σε train και test dataset και κάνουμε scale με τη χρήση της συνάρτησης StandardScaler()

In [14]:
X= dataset2.drop('y',axis=1)
y= dataset2['y']

X_train, X_test, y_train, y_test= train_test_split(X,y, test_size= 0.33, random_state= 4)

In [15]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train

**Βήμα 4ο**: Ορίζουμε τα κέντρα των νευρώνων με τη χρήση του k-means. Αρχικά τρέχει ο αλγόριθμος k-means για να βρούμε τα κέντρα και την τυπική απόκλιση των clusters.

In [17]:
K_cent= 8
km= KMeans(n_clusters= K_cent, max_iter= 100)
km.fit(X_train)
cent= km.cluster_centers_

In [18]:
max=0 
for i in range(K_cent):
	for j in range(K_cent):
		d= np.linalg.norm(cent[i]-cent[j])
		if(d> max):
			max= d
d= max

sigma= d/math.sqrt(2*K_cent)

Βήμα 5ο: Δημιουργούμε τον πίνακα σχέσεων F (γραμμικός συνδυασμός των Κ συναρτήσεων βάσης φj(x).

In [19]:
shape= X_train.shape
row= shape[0]
column= K_cent
F= np.empty((row,column), dtype= float)

In [20]:
for i in range(row):
  for j in range(column):
    dist= np.linalg.norm(X_train[i]-cent[j])
    F[i][j]= math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))

**Βήμα 6ο**: Βρίσκουμε τον πίνακα βάρους W για την εκπαίδευση του δικτύου

In [21]:
FTG= np.dot(F.T,F)
FTG_inv= np.linalg.inv(FTG)
fac= np.dot(FTG_inv,F.T)
W= np.dot(fac,y_train)

**Βήμα 7ο**: Ρύθμιση του πίνακα F για το test dataset

In [22]:
row= X_test.shape[0]
column= K_cent
F_test= np.empty((row,column), dtype= float)
for i in range(row):
	for j in range(column):
		dist= np.linalg.norm(X_test[i]-cent[j])
		F_test[i][j]= math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))

**Βήμα 8ο**: Ακρίβεια πρόβλεψης στο test dataset

In [ ]:
prediction= np.dot(F_test,W)
prediction= 0.5*(np.sign(prediction-0.5)+1)

score= accuracy_score(prediction,y_test)
print(score.mean())

## Ερωτήσεις

**Ερώτηση 1**: Να αναφέρετε επιγραμματικά με ποιον τρόπο λειτουργεί ένα νευρωνικό δίκτυο RBF

**Ερώτηση 2**:Με ποιούς τρόπους μπορούμε να υπολογίσουμε τα βάρη στα hidden layers; Ποιες διαφορές υπάρχουν στους τρόπους αυτούς; Ποιος παρέχει τη καλύτερη δυνατή λύση;

**Ερώτηση 3**: Επιχειρήστε να αλλάξετε την τιμή του k σε 10 και στη συνέχεια σε 50. Τι παρατηρείτε ως προς την ακρίβεια των αποτελεσμάτων;